# Posterior decoding

Posterior decoding consists
in picking state with the highest posterior for each position in the sequence independently; for 
each $i = 1,\ldots,N$:

\begin{equation}
y_i^* = \text{argmax}_{y_i \in \Lambda} P(Y_i=y_i | X = x).
\end{equation}

The **sequence posterior distribution** is the probability of a particular
hidden state sequence given that we have observed a particular
sequence. Moreover, we will be interested in two other posteriors distributions:
the **state posterior distribution**, corresponding to the
probability of being in a given state in a certain position given the
observed sequence; and the \textbf{transition posterior distribution},
which is the probability of making a particular transition, from position $i$ to
$i+1$, given the observed sequence. 

They are formally defined as follows:

- Sequence  Posterior
$$P(Y=y|X=x) = \frac{P(X=x,Y=y)}{P(X=x)}
$$

- State Posterior
$$
P(Y_i=y_i | X=x)
$$

- Transition Posterior
$$
P(Y_{i+1}=y_{i+1},Y_i=y_i| X=x)
$$


### Computing posteriors involves beeing able to compute $P(X=x)$
To compute the posteriors, a first step is to be able to compute the 
likelihood of
the sequence $P(X=x)$, which corresponds to summing the probability of all
possible hidden state sequences.

\begin{equation}
\mathbf{Likelihood\!:}\;\;\;\; P(X=x) = \displaystyle \sum_{y \in \Lambda^N} P(X=x,Y=y).
\end{equation}

The number of possible hidden state sequences is exponential in the
length of the sequence ($|\Lambda|^N$),
 which makes the sum over all of them hard. 
 In our simple
 example, there are $2^4 = 16$ paths, which we can actually explicitly enumerate
 and calculate their probability using Equation of the joint probability $P(x,y)$. But this is as far as it goes: for example, for Part-of-Speech
 tagging with a small tagset of 12 tags and a medium size
 sentence of length 10, there are $12^{10} = 61 917 364 224$ such
 paths. 
 
 Yet, we must be able to compute this sum (sum over $y \in \Lambda^N$) to compute the above likelihood
formula; this is called the inference problem. For sequence models, there is a well known dynamic programming algorithm,
the **Forward-Backward** (FB) algorithm, which allows the computation
to be performed in linear time, The runtime is linear with respect
to the sequence length. More precisely, 
the runtime is $O(N|\Lambda|^2)$. 
A naive enumeration would cost $O(|\Lambda|^N)$.

The FB algorithm relies on the independence of previous states
assumption, which  
is illustrated in the trellis view by having arrows only between consecutive states. 
The FB algorithm defines two auxiliary probabilities, the forward probability and the backward probability. 



## Efficient forward probability computation

The forward probability represents the probability that in position
$i$ we are in state $Y_i = c_k$ and that we have observed $x_1,\ldots,x_i$
up to that position. Therefore, its mathematical expression is:
\begin{equation}
\mathbf{Forward \ Probability\!:}\;\;\;\;  \mathrm{forward}(i, c_k) = P(Y_i = c_k, X_1=x_1,\ldots, X_i = x_i)
\end{equation}


Using the independence assumptions of the HMM we can compute $\mathrm{forward}(i, c_k)$ using all the forward computations \{$\mathrm{forward}(i -1, c)$ for $c \in \Lambda$\}. In order to facilitate the notation of the following argument we will denote by $x_{i:j}$  the assignemnt $X_i = x_i, \dots, X_j = x_j$. Therefore we can write   $\mathrm{forward}(i, y_i) $ as $P( y_i, x_{1:i } ) $ and rewrite the forward expression as follows:

\begin{equation}
  P( y_i, x_{1:i } ) =  \sum_{y_{i-1} \in \Lambda} P( y_i ,y_{i-1}, x_{1:i } )  =  \sum_{y_{i-1} \in \Lambda} P( x_i  | y_i,  y_{i-1},  x_{1:i-1 } ) \cdot P(y_i  | y_{i-1},  x_{1:i-1 }) \cdot P(y_{i-1},  x_{1:i-1 })  
\end{equation}


Using the **Observation independence** and the **Independence of previous states** properties of the first order HMM we have $P( x_i  | y_i,  y_{i-1},  x_{1:i-1 } ) = P( x_i  | y_i) $ and $P(y_i  | y_{i-1},  x_{1:i-1 })  = P(y_i  | y_{i-1})  $. Therefore the previous equation can be written, 
for $i \in \{2,\dots,N\}$ (where $N$ is the length of the sequence), as 

\begin{equation}
 \mathrm{forward}(i, y_i)  = \sum_{y_{i-1} \in \Lambda} P( x_i  | y_i, ) \cdot P(y_i  | y_{i-1}) \cdot \mathrm{forward}(i-1, y_{i-1})   
\end{equation}


The previous equation proves that  the forward probability can be defined by the
following recurrence rule: 

\begin{eqnarray}
\mathrm{forward}(1, c_k)&=& P_{\text{init}}(c_k|\text{start}) \times P_{\mathrm{emiss}}(x_1 | c_k)
 \\
 \mathrm{forward}(i, c_k) &=& \left(  \sum_{c_l \in \Lambda} P_{\mathrm{trans}}(c_k | c_l) \times \mathrm{forward}(i-1, c_l) \right) \times P_{\mathrm{emiss}}(x_i | c_k) 
 \\
  \mathrm{forward}(N+1, \text{stop}) &=& \sum_{c_l \in \Lambda} P_{\text{final}}(\text{ stop} | c_l) \times \mathrm{forward}(N, c_l).
\end{eqnarray}


Using the forward trellis one can compute the likelihood simply as:

\begin{equation}
P(X=x) = \mathrm{forward}(N+1, \text{ stop}).
\end{equation}

Although the forward probability is enough to calculate the likelihood of a given sequence, we will also need the backward probability to calculate the state posteriors. 




## Efficient backward probability computation



The backward probability is similar to the forward probability, but operates in the inverse direction.
It represents the probability of observing $x_{i+1},\ldots,x_N$ from position $i+1$ up to $N$, given that at position $i$ we are at state $Y_i = c_l$:

\begin{equation}
\mathbf{Backward \ Probability\!:}\;\;\;\;  \text{backward}(i, c_l) = P(X_{i+1}=x_{i+1},\ldots, X_N=x_N | Y_i = c_l).
\end{equation}



Using the independence assumptions of the HMM we can compute $\text{backward}(i, c_k)$ using all the backward computations $\text{backward}(i +1, c)$ for $c \in \Lambda$.

Therefore we can write   $\text{backward}(i, y_i) $ as $P( x_{i+1:N} | y_i ) $ and rewrite the forward expression as follows:

\begin{equation}
  P( x_{i+1:N} | y_i ) =  \sum_{y_{i+1} \in \Lambda} P( x_{i+1:N}, y_{i+1} | y_i)  =  \sum_{y_{i+1} \in \Lambda} P( x_{i+2:N} | y_i, y_{i+1}, x_{i+1}) 
   P( x_{i+1}, |  y_{i+1},  y_{i}) P( y_{i+1} | y_i)
\end{equation}

Using the previous equation we have proved that the backward probability can be defined by the following recurrence rule:


\begin{eqnarray}
\mathrm{backward}(N, c_l) &=& P_{\text{final}}(\text{stop} | c_l)  \\
\text{backward}(i, c_l) &=&  \displaystyle \sum_{c_k \in \Lambda} P_{\text{trans}}(c_k | c_l) \times 
\text{backward}(i+1, c_k) \times P_{\text{emiss}}(x_{i+1} | c_k) 
 \\
  \mathrm{backward}(0, \text{start}) &=& \sum_{c_k \in \Lambda} P_{\mathrm{init}}(c_k | \text{ start}) \times \mathrm{backward}(1, c_k) \times P_{\mathrm{emiss}}(x_{1} | c_k).
 \end{eqnarray}

Using the backward trellis one can compute the likelihood simply as:

\begin{equation}
P(X=x) = \mathrm{backward}(0, \text{start}).
\end{equation}



## The forward backward algorithm

We have seen how we can compute the probability of a sequence $x$ using the the forward and backward probabilities by computing  $\mathrm{forward}(N+1, \text{ stop})$ and $ \mathrm{backward}(0, \text{ start})$ respectively. Moreover,  the probability of a sequence $x$ can be computed with both forward and backward probabilities at a particular position $i$. 

The probability of a  given sequence $x$ at any position $i$ in the sequence can be computed
as follows:


\begin{eqnarray}
  P(X=x) &=& 
  \sum_{c_k \in \Lambda} P(X_1=x_1,\ldots, X_N=x_N,Y_i=c_k)\nonumber\\
  & =&
  \sum_{c_k \in \Lambda} 
  \underbrace{P(X_1=x_1,\ldots, X_i=x_i, Y_i=c_k)}_{\mathrm{forward}(i,c_k)} \times 
  \underbrace{P(X_{i+1}=x_{i+1},\ldots, X_N=x_N| Y_i=c_k)}_{\mathrm{backward}(i,c_k)}\nonumber\\
  &=& \sum_{c_k \in \Lambda} \mathrm{forward}(i,c_k) \times \mathrm{backward}(i,c_k).
\end{eqnarray}



This equation will work for any choice of $i$. Although redundant, this fact is useful when implementing an
HMM as a sanity check that the computations are being performed
correctly, since one can compute this expression for several $i$; they should all yield the same value. 

The following pseudocode shows the the forward backward algorithm. 

<img src="../images_for_notebooks/day_2/fb_alg.png"  style="max-width:100%; width: 50%">

The reader can notice that the $forward$ and $backward$ computations in the algorithm make use of $P_{emiss}$ and $P_{trans}$. There are a couple of details that should be taken into account if the reader wants to understand the algorithm using scores instead of probabilities.


- $forward(i,\hat{c})$  is computed using $P_{emiss}(x_i | \hat{c})$ which does not depend on the sum over all possible states $c_k \in  \Lambda $. Therefore when taking the logarithm of the sum over all possible states the recurrence of the forward computations can be split as a sum of two logarithms.


- $backward(i,\hat{c})$  is computed using $ P_{\text{trans}}(c_k | \hat{c} )$ and $P_{\text{emiss}}(x_{i+1} | c_k) $ both of  which  depend on $c_k$. Therefore when taking the logarithm of the sum the expression cannot be split as a sum of logarithms.



Given the forward and backward probabilities, one can compute both the state
and transition posteriors as follows:


\begin{align}
 \mathbf{State \ Posterior\!:}\;\;\;\;  & P(Y_i = y_i| X=x) = \frac{\mathrm{forward}(i, y_i) \times 
 \mathrm{backward}(i, y_i)}{P(X=x)}\\
 \mathbf{Transition \ Posterior\!:}\;\;\;\; &
 P(Y_i = y_i, Y_{i+1} = y_{i+1} | X=x)= \nonumber\\
 &
   \frac{\mathrm{forward}(i, y_i) \times 
   P_{\mathrm{trans}}(y_{i+1}|y_i) \times
   P_{\mathrm{emiss}}(x_{i+1}|y_{i+1}) \times
 \mathrm{backward}(i+1, y_{i+1})}{P(X=x)}
\end{align}



A graphical representation of these posteriors is illustrated in the following figure:

<img src="../images_for_notebooks/day_2/ex_trellis.png"  style="max-width:100%; width: 80%">

On the left it is shown that $\mathrm{forward}(i, y_i)  \times \mathrm{backward}(i, y_i)$ returns the sum of all paths that contain the state $y_i$, weighted by $P(X=x)$; on the right we can see that 

$$\mathrm{forward}(i, y_i) \times P_{\mathrm{trans}}(y_{i+1}|y_i) \times P_{\mathrm{emiss}}(x_{i+1}|y_{i+1}) \times \mathrm{backward}(i+1, y_{i+1})$$

returns the same for all paths containing the edge from $y_i$ to $y_{i+1}$. Thus, these posteriors can be seen as the ratio of the number of paths that contain the given state or transition (weighted by $P(X=x)$) and the number of possible paths in the graph marginal.

As a practical example, given that the person performs the sequence of actions $\text{ walk} \text{ walk} \text{ shop} \text{ clean}$, we want to know the probability of having been raining in the second day. The state posterior probability for this event can be seen as the probability that the sequence of actions above was generated by a sequence of weathers and where it was raining in the second day. In this case, the possible sequences would be all the sequences which have ${\tt rainy}$ in the second position.


Using the state posteriors, we are ready to perform posterior
decoding. 
The strategy is to compute the state posteriors 
for each position $i \in \{1,\ldots,N\}$
and each state $c_k \in \Lambda$, and 
then pick the arg-max at each position:

$$
{\widehat y_i} := \text{argmax}_{y_i \in \Lambda} P(Y_i=y_i| X=x).
$$


## About the hmm class

The HMM class inherits from SequenceClassifier

In the following exercise we will use the run_forward function which 

    def run_forward(self, initial_scores, transition_scores, final_scores, emission_scores):
            length = np.size(emission_scores, 0) # Length of the sequence.
            num_states = np.size(initial_scores) # Number of states.

            # Forward variables.
            forward = np.zeros([length, num_states]) + logzero()

            # Initialization.
            forward[0,:] = emission_scores[0,:] + initial_scores

            # Forward loop.
            for pos in xrange(1,length):
                for current_state in xrange(num_states):
                    # Note the fact that multiplication in log domain turns a sum and sum turns a logsum
                    forward[pos, current_state] = \
                            logsum(forward[pos-1, :] + transition_scores[pos-1, current_state, :])
                    forward[pos, current_state] += emission_scores[pos, current_state]

            # Termination.
            log_likelihood = logsum(forward[length-1,:] + final_scores)

            return log_likelihood, forward

## Exercise 2.5 

Run the provided forward-backward algorithm on the first train sequence.
Observe that both the forward and the backward passes give the same log-likelihood.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
# We will this append to ensure we can import lxmls toolking
sys.path.append('../../lxmls-toolkit')

In [25]:
import lxmls.readers.simple_sequence as ssr
simple = ssr.SimpleSequence()

simple.train.seq_list

[walk/rainy walk/sunny shop/sunny clean/sunny ,
 walk/rainy walk/rainy shop/rainy clean/sunny ,
 walk/sunny shop/sunny shop/sunny clean/sunny ]

In [26]:
import lxmls.sequences.hmm as hmmc
import lxmls.readers.simple_sequence as ssr
simple = ssr.SimpleSequence()

hmm = hmmc.HMM(simple.x_dict, simple.y_dict)
hmm.train_supervised(simple.train)
initial_scores, transition_scores, final_scores, emission_scores = hmm.compute_scores(simple.train.seq_list[0])

../../lxmls-toolkit/lxmls/sequences/hmm.py:194: RuntimeWarning: divide by zero encountered in log
  transition_scores[pos-1, :, :] = np.log(self.transition_probs)
../../lxmls-toolkit/lxmls/sequences/hmm.py:192: RuntimeWarning: divide by zero encountered in log
  emission_scores[pos, :] = np.log(self.emission_probs[sequence.x[pos], :])
../../lxmls-toolkit/lxmls/sequences/hmm.py:197: RuntimeWarning: divide by zero encountered in log
  final_scores = np.log(self.final_probs)


In [27]:
hmm.state_labels

{'rainy': 0, 'sunny': 1}

In [28]:
hmm.observation_labels

{'clean': 2, 'shop': 1, 'tennis': 3, 'walk': 0}

In [55]:
initial_scores

array([-0.40546511, -1.09861229])

In [58]:
np.exp(emission_scores)

array([[ 0.75 ,  0.25 ],
       [ 0.75 ,  0.25 ],
       [ 0.25 ,  0.375],
       [ 0.   ,  0.375]])

In [75]:
emission_scores

array([[-0.28768207, -1.38629436],
       [-0.28768207, -1.38629436],
       [-1.38629436, -0.98082925],
       [       -inf, -0.98082925]])

In [76]:
initial_scores

array([-0.40546511, -1.09861229])

In [78]:
final_scores

array([       -inf, -0.98082925])

In [80]:
transition_scores

array([[[-0.69314718,        -inf],
        [-0.69314718, -0.47000363]],

       [[-0.69314718,        -inf],
        [-0.69314718, -0.47000363]],

       [[-0.69314718,        -inf],
        [-0.69314718, -0.47000363]]])

In [87]:
log_likelihood, forward = hmm.decoder.run_forward(initial_scores, transition_scores,final_scores, emission_scores)
print "forward:\n", forward, "\n"
print '\n Log-Likelihood =', log_likelihood

forward:
[[-0.69314718 -2.48490665]
 [-1.67397643 -2.58334672]
 [-3.75341798 -2.94017562]
 [       -inf -4.08740307]] 


 Log-Likelihood = -5.06823232601


In [89]:
log_likelihood, backward = hmm.decoder.run_backward(initial_scores, transition_scores, final_scores, emission_scores)
print "backward :\n", backward, "\n"
print 'Log-Likelihood =', log_likelihood

backward :
[[-4.41863845 -5.73879301]
 [-3.67819455 -3.88249502]
 [-2.65480569 -2.43166214]
 [       -inf -0.98082925]] 

Log-Likelihood = -5.06823232601


# Exercise 2.6 

Compute the node posteriors for the first training sequence (use the provided compute posteriors function), 
and look at the output. Note that the state posteriors are a proper probability distribution 
(the lines of the result sum to 1).

In [32]:
import lxmls.sequences.hmm as hmmc
import lxmls.readers.simple_sequence as ssr
simple = ssr.SimpleSequence()

hmm = hmmc.HMM(simple.x_dict, simple.y_dict)
hmm.train_supervised(simple.train)
initial_scores, transition_scores, final_scores, emission_scores = hmm.compute_scores(simple.train.seq_list[0])
state_posteriors, transition_posteriors, log_likelihood = hmm.compute_posteriors(initial_scores, transition_scores, final_scores, emission_scores)

In [33]:
print state_posteriors

[[ 0.95738152  0.04261848]
 [ 0.75281282  0.24718718]
 [ 0.26184794  0.73815206]
 [ 0.          1.        ]]


#  Exercise 2.7 

Run the posterior decode on the first test sequence, and evaluate it.

In [34]:
simple = ssr.SimpleSequence()

hmm = hmmc.HMM(simple.x_dict, simple.y_dict)
hmm.train_supervised(simple.train)
initial_scores, transition_scores, final_scores, emission_scores = hmm.compute_scores(simple.train.seq_list[0])

y_pred = hmm.posterior_decode(simple.test.seq_list[0  ])
print "Prediction test 0:\n\t", y_pred, "\n"
print "Truth test 0:\n\t", simple.test.seq_list[0]


Prediction test 0:
	walk/rainy walk/rainy shop/sunny clean/sunny  

Truth test 0:
	walk/rainy walk/sunny shop/sunny clean/sunny 


Do the same for the second test sentence

In [35]:
y_pred = hmm.posterior_decode(simple.test.seq_list[1])
# There are nan values in the backward and forward probabilites caused by
# not having observed tennis

print "Prediction test 1:"
print y_pred
print "Truth test 1:"
print simple.test.seq_list[1]

Prediction test 1:
clean/rainy walk/rainy tennis/rainy walk/rainy 
Truth test 1:
clean/sunny walk/sunny tennis/sunny walk/sunny 


../../lxmls-toolkit/lxmls/sequences/sequence_classifier.py:78: RuntimeWarning: invalid value encountered in subtract
  state_posteriors[pos,:] -= log_likelihood
../../lxmls-toolkit/lxmls/sequences/sequence_classifier.py:91: RuntimeWarning: invalid value encountered in double_scalars
  transition_posteriors[pos, state, prev_state] -= log_likelihood


What is wrong?

**Note the observations for the second test sequence: the observation tennis was never seen at training time**, so the probability for it will be zero (no matter what state). This will make all possible state sequences have zero probability. As seen in the previous lecture, this is a problem with generative models, which can be corrected using smoothing (among other options).


Change the train supervised method to add smoothing:
```
   def train_supervised(self,sequence_list, smoothing):
```

In [36]:
hmm.train_supervised(simple.train, smoothing=0.1)
y_pred = hmm.posterior_decode(simple.test.seq_list[0])
print "Prediction test 0 with smoothing:"
print "\t",y_pred 
print "Truth test 0:"
print "\t",simple.test.seq_list[0]
y_pred = hmm.posterior_decode(simple.test.seq_list[1])
print "\n"
print "Prediction test 1 with smoothing:"
print "\t",y_pred
print "Truth test 1:"
print "\t",simple.test.seq_list[1]

Prediction test 0 with smoothing:
	walk/rainy walk/rainy shop/sunny clean/sunny 
Truth test 0:
	walk/rainy walk/sunny shop/sunny clean/sunny 


Prediction test 1 with smoothing:
	clean/sunny walk/sunny tennis/sunny walk/sunny 
Truth test 1:
	clean/sunny walk/sunny tennis/sunny walk/sunny 
